In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
import MathOptInterface as MOI
import Ipopt 
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
using LinearAlgebra
using Plots
using Random
using JLD2
using Test
import MeshCat as mc 
using Statistics
using ControlSystems
using SparseArrays
using OSQP
using Pkg
using PyCall
using Printf
Pkg.build("PyCall")

project_dir = pwd()
font_path = joinpath(project_dir, "font")
pushfirst!(PyVector(pyimport("sys")."path"), font_path)
@pyimport letters as letters

  Activating environment at `~/Courses/Optimal_Control/Final_Project/Project.toml`
    Building Conda ─→ `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/51cab8e982c5b598eea9c8ceaced4b58d9dd37c9/build.log`
    Building PyCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/9816a3826b0ebf49ab4926e2b18842ad8b5c8f04/build.log`


In [2]:
include(joinpath(@__DIR__, "utils","quadrotor.jl"))

function discrete_dynamics(params::NamedTuple, x::Vector, u)
    # discrete dynamics
    # x - state 
    # u - control 
    # dt comes from params.model.dt 
    return rk4(params.model, quadrotor_dynamics, x, u, params.model.dt)
end

discrete_dynamics (generic function with 1 method)

### Problem Initialization

In [3]:
# problem size 
nx = 12
nu = 4
dt = 0.05 
tf = 10
t_vec = 0:dt:tf 
N = length(t_vec)

nt = Int(tf/dt)+1    # number of time steps
thist = Array(range(0,dt*(nt-1), step=dt));

# tracking cost function
Q = 1*diagm([1*ones(3);.1*ones(3);1*ones(3);.1*ones(3)])
R = .1*diagm(ones(nu))
Qf = 10*Q 

# dynamics parameters (these are estimated)
m = 0.5
g = 9.81
model = (mass=m,
        J=Diagonal([0.0023, 0.0023, 0.004]),
        gravity=[0,0,-g],
        L=0.1750,
        kf=1.0,
        km=0.0245,dt = dt)


# the params needed by iLQR 
params = (
    N = N, 
    nx = nx, 
    nu = nu, 
    Q = Q, 
    R = R, 
    Qf = Qf, 
    model = model
)

# Linearized hovering dynamics
x_hover = zeros(nx)
u_hover = [0.25*m*g; 0.25*m*g;0.25*m*g; 0.25*m*g]

# Find A and B matrix
A = FD.jacobian(dx->discrete_dynamics(params,dx,u_hover),x_hover);
B = FD.jacobian(du->discrete_dynamics(params,x_hover,du),u_hover);

#Thrust limits
umin = [0.2*m*g; 0.2*m*g; 0.2*m*g; 0.2*m*g];
umax = [0.6*m*g; 0.6*m*g; 0.6*m*g; 0.6*m*g];


# Cost weights
Q = Array(2.0*I(nx));
R = Array(.01*I(nu));
Qn = Array(1.0*I(nx));

In [4]:
goal_positions = letters.Letters(text="OCRL", height=2, start=[0,0,0], viz=false).positions
n_drones = size(goal_positions)[1]

26

In [5]:
#Build QP matrices for OSQP
nh = 20 #one second horizon at 20Hz
U = kron(Diagonal(I,nh), [I zeros(nu,nx)]) #Matrix that picks out all u
Θ = kron(Diagonal(I,nh), [0 0 0 0 1 0 0 0]) #Matrix that picks out all x3 (θ)

# Big [R1 0 0, 0 Q2 0, ....]matrix also used for LQR
H = sparse([kron(Diagonal(I,nh-1),[R zeros(nu,nx); zeros(nx,nu) Q]) zeros((nx+nu)*(nh-1), nx+nu); zeros(nx+nu,(nx+nu)*(nh-1)) [R zeros(nu,nx); zeros(nx,nu) Qn]])
b = zeros((n_drones, nh*(nx+nu)))
b = [b[i,:] for i in 1:size(b,1)]

# [AB-I, AB-I, ...] also used for LQR
C = sparse([[B -I zeros(nx,(nh-1)*(nu+nx))]; zeros(nx*(nh-1),nu) [kron(Diagonal(I,nh-1), [A B]) zeros((nh-1)*nx,nx)] + [zeros((nh-1)*nx,nx) kron(Diagonal(I,nh-1),[zeros(nx,nu) Diagonal(-I,nx)])]])

# Dynamics + Thrust limit constraints
min_dist = .5
D = [C; U]

lb = repeat([zeros(nx*nh); kron(ones(nh),umin-u_hover)]', outer = n_drones)
lb = [lb[i,:] for i in 1:size(lb,1)]

ub = repeat([zeros(nx*nh); kron(ones(nh),umax-u_hover)]', outer = n_drones)
ub = [ub[i,:] for i in 1:size(ub,1)]

# Dynamics + thrust limit + bound constraint on θ to keep the system within small-angle approximation
# D = [C; U; Θ]
# lb = [zeros(nx*nh); kron(ones(nh),umin-u_hover); -0.2*ones(nh)]
# ub = [zeros(nx*nh); kron(ones(nh),umax-u_hover); 0.2*ones(nh)]

probs = OSQP.Model[]
for i = 1:n_drones
    prob =  OSQP.Model()
    err1 = OSQP.setup!(prob; P=H, q=b[i], A=D, l=lb[i], u=ub[i], verbose=true)
    push!(probs, prob)
end


-----------------------------------------------------------------
           OSQP v0.6.2  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2021
-----------------------------------------------------------------
problem:  variables n = 320, constraints m = 320
          nnz(P) + nnz(A) = 1774
settings: linear system solver = qdldl,
          eps_abs = 1.0e-03, eps_rel = 1.0e-03,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off, time_limit: off

-----------------------------------------------------------------
           OSQP v0.6.2  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 

In [6]:
#MPC Controller
function mpc_controller(t,X,Xref)
    Δu = zeros((n_drones, nu))

    for i = 1:n_drones
        #Update QP problem
        lb[i][1:12] .= -A*X[i]
        ub[i][1:12] .= -A*X[i]

        for j = 1:(nh-1)
            b[i][(nu+(j-1)*(nx+nu)).+(1:nx)] .= -Q*Xref[i]
        end

        b[i][(nu+(nh-1)*(nx+nu)).+(1:nx)] .= -Qn*Xref[i]

        OSQP.update!(probs[i], q=b[i], l=lb[i], u=ub[i])

        #Solve QP
        results = OSQP.solve!(probs[i])

        Δu[i,:] = u_hover + results.x[1:nu]
    end

    Δu = [Δu[i,:] for i in 1:size(Δu,1)]
    return Δu
end

mpc_controller (generic function with 1 method)

In [7]:
function closed_loop(X0, controller,N)
    
    xhist = zeros(n_drones,nx,N)

    U0 = controller(1,X0)
    uhist = zeros(n_drones,nu,N-1)

    for i = 1:n_drones
        uhist[i,:,1] .= U0[i]
        xhist[i,:,1] .= X0[i]
    end
    
    for k = 1:(N-1)
        Xn = xhist[:,:,k]
        Xn = [Xn[i,:] for i in 1:size(Xn,1)]

        Uk = controller(k,Xn)
        for i = 1:n_drones 
            uhist[i,:,k] = max.(min.(umax, Uk[i]), umin) #enforce control limits
            xhist[i,:,k+1] .= discrete_dynamics(params,xhist[i,:,k],uhist[i,:,k])  # TODO Change so real and estimated model are different
        end
    end
    return xhist, uhist
end

closed_loop (generic function with 1 method)

In [8]:
# goal_positions = letters.Letters(text="O", height=2, start=[0,0,0], viz=false).positions
# n_drones = size(goal_positions)[1]

goal_matrix = zeros(n_drones,12)
goal_matrix[:,1:3] = 2*goal_positions
goal_matrix[:,1] = goal_matrix[:,1] .- 10
goal_matrix = [goal_matrix[i,:] for i in 1:size(goal_matrix,1)]

start_matrix = zeros(n_drones,12)
start_matrix[:,1:3] = 2*randn((n_drones,3))
start_matrix[:,3]   = start_matrix[:,3] .+ 2
start_matrix = [start_matrix[i,:] for i in 1:size(start_matrix,1)]

xhist, uhist = closed_loop(start_matrix, (t,x)->mpc_controller(t,x,goal_matrix), nt);


iter   objective    pri res    dua res    rho        time
   1  -9.0388e+02   4.23e+00   8.66e+02   1.00e-01   3.45e-04s
  75  -3.6956e+02   2.04e-04   1.85e-03   1.18e+00   9.57e-04s

status:               solved
number of iterations: 75
optimal objective:    -369.5575
run time:             9.63e-04s
optimal rho estimate: 9.80e-01

iter   objective    pri res    dua res    rho        time
   1  -1.1261e+03   4.48e+00   8.94e+02   1.00e-01   3.13e-04s
  50  -5.7723e+02   3.10e-03   1.30e-02   1.12e+00   7.25e-04s

status:               solved
number of iterations: 50
optimal objective:    -577.2292
run time:             7.30e-04s
optimal rho estimate: 1.38e+00

iter   objective    pri res    dua res    rho        time
   1  -1.4562e+03   5.22e+00   1.04e+03   1.00e-01   2.66e-04s
  50  -5.3652e+00   3.78e-03   1.64e-02   1.28e+00   6.52e-04s

status:               solved
number of iterations: 50
optimal objective:    -5.3652
run time:             6.57e-04s
optimal rho estimate: 1.53e+0

In [9]:
x = []
for k = 1:n_drones
    xhist_k = xhist[k,:,:]
    xhist_k = [xhist_k[:,i] for i in 1:size(xhist_k,2)]
    push!(x, xhist_k)
end

display(animate_quadrotor_n(x,x,n_drones, params.model.dt))

┌ Info: Listening on: 127.0.0.1:8700, thread id: 1
└ @ HTTP.Servers /home/acantu/.julia/packages/HTTP/vnQzp/src/Servers.jl:382
┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /home/acantu/.julia/packages/MeshCat/I6NTX/src/visualizer.jl:63


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("24" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6d, 0x65, 0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x32, 0x34], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "4" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x6d, 0x65, 0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x34], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "1" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x6d, 0x65, 0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x31], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "12" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6d, 0x65, 0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x31, 0x32], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "20" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6d, 0x65, 0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x32, 0x30], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "2" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x6d, 0x65, 0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x32], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "6" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x6d, 0x65, 0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x36], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "25" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6d, 0x65, 0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x32, 0x35], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "23" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6d, 0x65, 0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x32, 0x33], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "22" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6d, 0x65, 0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x32, 0x32], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}())…)))), Set{HTTP.WebSockets.WebSocket}(), ip"127.0.0.1", 8700))

## Scheduler to avoid Collisions

In [ ]:
min_dist = 0.7
function all_scheduled(start_times)
    for i = 1:n_drones
        if(start_times[i] < 0)
            return false
        end
    end
    return true
end

function all_scheduled_or_blocked(start_times)
    for i = 1:n_drones
        if(start_times[i] == -1)
            return false
        end
    end
    return true
end

# n: drone number
# t: current time
function collision(params, start_t, paths, n, t, ndrones)
    for i = 1:ndrones

        collides = false
        if i == n
            continue
        end
        if start_t[i] >= 0  # If drone i was alrady scheduled
            
            n_path_o = mapreduce(permutedims, vcat, paths[i])
            n_path_o = n_path_o[:,1:3]

            # n_path = n_path_o[N,1]'.*ones((N,3))
            # n_path[1:N-start_t[i]+1,:] = n_path_o[start_t[i]:N,:]

            n_path = n_path_o[N,1]'.*ones((N,3))
            n_path[1:N-(t-start_t[i]),:] = n_path_o[t-start_t[i]+1:N,:]

            for k = 1:N
                obst_pos = n_path[k,:]
                dron_pos = paths[n][k][1:3]

                dist = norm(obst_pos - dron_pos)

                if dist < min_dist
                    @printf("DRONE %d: Collision Detected at time %f with drone #%d\n", n, (t+k)*dt, i)
      
                    collides = true
                    return true
                end
            end
        end
    end

    return false
end

# function path_scheduler(params, paths)
#     t = 0
#     start_times = -1* ones(n_drones)
#     start_time[1] = t

#     while !all_scheduled_or_blocked(start_times)
#         for i = 1:n_drones
#             if start_times[i] < 0 
#                 display(i)
#             end
#         end
#     end
# end

In [ ]:
n_drones_t = n_drones
curren_time = 1
drone_to_test = 1
st = Int.(ones(n_drones))
st[1] = -1
collision(params,st, x, drone_to_test, curren_time, n_drones_t)